# CS 506 Spring 2021 - HW3
## Social Networks and Recommendataion Systems
#### Total: 25 Points

##### Package Limitations: None

### 1. Background

In this homework, you will try to recommend new collaborations to researchers of the Machine Learning community. Our approach will follow the guidelines of collaborative filtering: “**If your past behavior/preferences were similar to some other user’s, your future behavior may be as well**”. As an example, imagine you like Rolling Stones, Beatles and Jimmy Hendrix. It turns out that most people that like the aforementioned artists, are also fans of Eric Clapton. Then, it is very likely that if you listen to Eric Clapton’s music, you will like it as well.

In this assignment you will implement a **collaborative filtering recommendation system** for suggesting new collaborations to Machine Learning researchers.

**A network as a graph**: A graph or network represents relationships among different entities (users of a social network, researchers, products, etc.). Those entities are represented as nodes and the relationships between them (friends on Facebook, co-authors of a research paper, products purchased together) as edges. When there is an edge between two nodes, x and y, we say that y is a neighbor (or friend) of x (and also - as the graphs we consider are undirected - x is also a neighbor of y).

**Representing a graph in Python**: A widely used library in Python, for representing graphs is [NetworkX](https://networkx.github.io/documentation/stable/). You can read the documentation for more information on how to use this library.

### 2. Recommend new collaborations - The ML Community case 

In order to provide new collaborations and test the efficiency of the methods used, you are given two files (you can find them on piazza):

- ”old edges.txt”: In this file, every line contains the names of two re- searchers that have co-authored a paper in one of the top Machine Learn- ing conferences (NeurIPS, ICLR, ICML) between 2010 and 2016.
- ”new edges.txt”: In this file, every line contains the names of two re- searchers (from those existing in the above file) that formed a new (non- existing before) collaboration, in either 2017 and 2018.

With the first file in hand, you will answer the following question:
“For author X, list some non-collaborators in order, starting with the best col- laborator recommendation and ending with the worst”. A non-friend is a user who is not X and is not a collaborator of X. Depending on the recommendation algorithm you are going to choose, the list may include all non-collaborators or some of them.

Then, using the second file, with actual new collaborations formed in the next 3 years, you will test the efficiency of these algorithms.


### Tasks
a) [3 pts.] Write a function that reads the file “old edges.txt” and create a graph using NetworkX. (This is a tab-separated value (TSV) file, you may use packages such as Pandas to read it. )


In [1]:
# You can add functions, inputs outputs to existing functions. 
# Please do NOT change name of the existing functions

from typing import Tuple, List, Dict, Callable
import networkx as nx

def read_old_edges(file_path: str) -> nx.Graph:
    """
    read old edges text file and return a NetworkX graph
    
    :param file_path: string file path to old_edges.txt
    :return: network graph instance of the graph
    """
    # graph = instance of nx graph
    graph = nx.Graph()
    graph = nx.read_edgelist(file_path,
                             delimiter= "\t",
                             nodetype = str)
    
    return graph

b) [3 pts.] Write a function that reads the file “new edges.txt” and for each author, keeps track of the new collaborations this user formed during 2017-2018.


In [2]:
def read_new_edges(file_path: str):
    """
    read new edges text file
    
    :param file_path: string file path to new_edges.txt
    :return: (you decide)
    """
    graph = read_old_edges(file_path)
    nodes = graph.nodes()
    arr = [[node] + list(graph.neighbors(node)) for node in nodes]
    return arr

In [3]:
old = read_old_edges('old_edges.txt')
new = read_new_edges('new_edges.txt')

In 2017 and 2018, there were 1,757 new edges formed between existing authors. For the next tasks, pick (and recommend new collaborations for) those authors that formed at least 10 new connections between 2017-2018. In the remaining, when we talk about author X, we refer to one of those authors.

c) [5 pts.] **Recommend by number of common friends**

The intuition behind this recommendation algorithm is that if non-friend Y is your friend’s friend, then maybe Y should be your friend too. If person Y is the friend of many of your friends, then Y is an even better recommendation. 

In [4]:
def get_network(graph, person):
    return set(graph.neighbors(person))

def get_peoples(graph, author):
    res = []
    for person in graph.nodes():
        if person != author:
            res.append((author,person))
    return res

def get_commonality(graph, net, author):    
    common = {}
    for person in net:
        a = get_network(graph, author)
        p = get_network(graph, person)
        num = len( (a).intersection(p) )
        if num >= 1:
            common[person] = num
    # sort dict based on values but keep keys: learned using link below        
    # https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
    arr = list(sorted(common.items(), key=lambda item: item[1], reverse=True))
    res = [item[0] for item in arr] 

    return res

In [5]:
def common_friends_number(graph: nx.Graph, author) -> List[str]:
    """
    Return list of authors who have a common neighbor as 
    given author sorted by number of common friends. 
    
    :param graph: collaboration graph in nextworkX
    :return: list of new collaborators' name to recommend
    """
    network = get_network(graph, author)    
    net = set()
    for person in network:
        net.update(name for name in (get_network(graph, person) - network))
    net -= {author}
    
    res = get_commonality(graph, net, author)
    return res

# common_friends_number(old, 'Pradeep Ravikumar')

d) [5 pts.] **Make recommendations using Jaccard’s Index**

If Γ(X) is the set of neighbors of X, then the metric we used in part (c), assigns to a non-friend y, the following recommendation score (with respect to X): score(y) = |Γ(X)∩Γ(y)|. Jaccard’s Index scales this score by taking into account the union of X and Y ’s neighbors. Intuitively, X and Y are more similar, if what they have in common is as close as possible to what they have together.


In [6]:
def sort_solution(solution, network):
    
    ans = sorted([[a, b, c] for a, b, c in solution if c > 0], 
                 key=lambda item: item[2], reverse=True)

    result = [person[1] for person in ans 
              if person[1] not in network]
    
    return result

In [7]:
def jaccard_index(graph: nx.Graph, author) -> List[str]:
    """
    Return list of authors who have a common neighbor as 
    given author sorted by Jaccard Index (see pdf for equation) 
    
    :param graph: collaboration graph in nextworkX
    :return: list of new collaborators' name to recommend
    """
    people = get_peoples(graph, author)
    network = list(get_network(graph, author)) + [author]
    jaccard = nx.jaccard_coefficient(graph, people)
    return sort_solution(jaccard, network)

# jaccard_index(old, 'Michael I. Jordan')

e)  [5 pts.] **Make recommendations using Adamic/Adar Index**

For part (c), we made recommendations using common neighbors. However, when assigning a score to Y , instead of just taking a count of the number of common neighbors, we take a weighted sum of them, where the weight of each common neighbor of X and Y , call her Z, is the inverse of the logarithm of the number of Z’s neighbors. In that way, we value more common neighbors that are more selective.

In [8]:
def adamic_adar_index(graph: nx.Graph, author) -> List[str]:
    """
    Return list of recommendations of a given author sorted 
    by Adamic / Adar Index (see pdf for equation) 
    
    :param graph: collaboration graph in nextworkX
    :return: list of new collaborators' name to recommend
    """
    people = get_peoples(graph, author)
    network = list(get_network(graph, author)) + [author]
    adar = nx.adamic_adar_index(graph, people) 
    return sort_solution(adar, network)
    
# adamic_adar_index(old, 'Michael I. Jordan')

f) [4 pts.] **How good are the recommendations we make?** 

Previously, you implemented 3 functions, that given a user X provide recommendations for this user. In this task, you will check how good these recommendations are using the actual new connections formed during 2017-2018.

You will use two different ways, to calculate the efficiency of every approach:

- For each user X, take the 10 first recommendations for this user, and calculate the number of them that were actually formed during 2017-2018. You should report the average among users X.

- For each newly formed collaboration of user X, calculate the rank of this collaboration (the index where this new node Y appears in the recommendations list for X). Report the average among newly formed edges.

In [9]:
# def top_k_recommendation_accuracy(graph: nx.Graph, index_method: Callable, new_edges) -> float:
#     """
#     Among top k recommendations of every user, return average number 
#     of recommendations that becomes reality
    
#     :param graph: collaboration graph in nextworkX
#     :param index_method: function that is used to make recommendation
#     :param new_edges: (you decide)
#     :return: average accuracy of predictions among all users
#     """
#     raise NotImplementedError("Implement this!")
    
    
# def new_collaboration_rank(graph: nx.Graph, index_method: Callable, new_edges) -> float:
#     """
#     Among the new collaborations of every user, return average rank 
#     of the collaboration calculated by the function
    
#     :param graph: collaboration graph in nextworkX
#     :param index_method: function that is used to make recommendation
#     :param new_edges: (you decide)
#     :return: average accuracy of predictions among all users
#     """
#     raise NotImplementedError("Implement this!")

In [10]:
def get_old(old):
    res = []
    for person in dict(old).keys():
        res.append(person)
    return res

def get_new(new):
    res = []
    for person in dict(new).keys():
        value = new[person]
        if value >= 10:
            res.append(person)
    return res

def get_intersection(old_degree, new_degree):
    old = get_old(old_degree)
    new = get_new(new_degree)
    both = set(new).intersection(set(old))
    return both

In [25]:
def get_topk_avg(graph, new_edges, common, index_method):
    counter = 0
    for friend in common:
        # top 10 and neighbors and find intersection between both
        top_10 = set(index_method(graph, friend)[:10])
        neighbor = get_network(new_edges, friend)
        similar = (top_10).intersection(neighbor)
        counter += len(similar) / len(neighbor)  
    return counter/len(common) 

def top_k_recommendation_accuracy(graph: nx.Graph, index_method: Callable, new_edges) -> float:
    """
    Among top k recommendations of every user, return average number 
    of recommendations that becomes reality
    
    :param graph: collaboration graph in nextworkX
    :param index_method: function that is used to make recommendation
    :param new_edges: (you decide)
    :return: average accuracy of predictions among all users
    """
    # inputs to degree
    old_degree = graph.degree()
    new_degree = new_edges.degree()
    
    # degree to array
    common = get_intersection(old_degree, new_degree)
    return get_topk_avg(graph, new_edges, common, index_method)


In [27]:
def get_index_avg(graph, new_edges, both, index_method):
    position = {}
    for person in both:
        friends = index_method(graph, person)
        n = len(friends)
        neighbor = get_network(new_edges, person)
        
        for people in neighbor:
            if people in friends:
                position[people] = friends.index(people) 
            else: position[people] = n+1 
            
    avg = sum(position.values()) / len(position.values())
    return avg

def new_collaboration_rank(graph: nx.Graph, index_method: Callable, new_edges) -> float:
    """
    Among the new collaborations of every user, return average rank 
    of the collaboration calculated by the function
    
    :param graph: collaboration graph in nextworkX
    :param index_method: function that is used to make recommendation
    :param new_edges: (you decide)
    :return: average accuracy of predictions among all users
    """
    
    old_degree = graph.degree()
    new_degree = new_edges.degree()
    
    # degree to array
    both = get_intersection(old_degree, new_degree)

    return get_index_avg(graph, new_edges, both, index_method)
    

In [26]:
new_edges = read_old_edges('new_edges.txt')
common = top_k_recommendation_accuracy(graph = old, new_edges = new_edges, index_method = common_friends_number)
jac = top_k_recommendation_accuracy(graph = old, new_edges = new_edges, index_method = jaccard_index)
ada = top_k_recommendation_accuracy(graph = old, new_edges = new_edges, index_method = adamic_adar_index)


print('top_k_recommendation_accuracy:')
print()
print('common_friends_number:', common)
print('jaccard_index:', jac)
print('adamic_adar_index', ada)

top_k_recommendation_accuracy:

common_friends_number: 0.07443102996829495
jaccard_index: 0.0502155346520294
adamic_adar_index 0.0675579020264892


In [29]:
rank_common = new_collaboration_rank(graph = old, new_edges = new_edges, index_method = common_friends_number)
rank_jac = new_collaboration_rank(graph = old, new_edges = new_edges, index_method = jaccard_index)
rank_ada = new_collaboration_rank(graph = old, new_edges = new_edges, index_method = adamic_adar_index)

print('new_collaboration_rank:')
print()
print('common_friends_number:', rank_common)
print('jaccard_index:', rank_jac)
print('adamic_adar_index', rank_ada)

new_collaboration_rank:

common_friends_number: 97.4074074074074
jaccard_index: 98.89711934156378
adamic_adar_index 97.00411522633745


e) [**Bonus Question**] [2 pts.]
Doing some literature search, suggest your own algorithm for recommend- ing new links to a user X. Argue about the choice you make, why it makes sense to suggest users that way? How is the efficiency of this algorithm, compared to the ones you implemented in parts (c), (d) and (e)?